# download libraries


In [ ]:
# Requirements:
!pip install rtdl
!pip install libzero==0.0.4 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install openml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 119 kB 7.9 MB/s 
     |████████████████████████████████| 76 kB 3.5 MB/s 
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137326 sha256=03c400ac038fdf67e4ea60f0a4b6754aee130d36ab24f0dcb491746f7fdf0365
  Stored in directory: /root/.cache/pip/wheels/6a/20/88/cf4ac86aa18e2cd647ed16ebe274a5dacee9d0075fa02af250
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=bfa8b30346f830ca024e303fb08d71723a00dd92a57e799a54973970157c387c
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml liac-arff


In [ ]:
!pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 6.9 MB/s 
     |████████████████████████████████| 181 kB 56.8 MB/s 
     |████████████████████████████████| 157 kB 66.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 157 kB 63.9 MB/s 
     |████████████████████████████████| 157 kB 54.4 MB/s 
     |████████████████████████████████| 157 kB 62.1 MB/s 
     |████████████████████████████████| 157 kB 54.6 MB/s 
     |████████████████████████████████| 156 kB 57.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e8fffbbcf70ce2deef1c117046224934c867a29cd4117f9443fb069857657872
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# Requirements:
!pip install rtdl
!pip install libzero==0.0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.1 MB/s 


#Auxiliary classes 

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable

import numpy as np
import openml
import pandas as pd
from openml import OpenMLDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

task_ids = (
    40588,
    40589,
    40590,
    40591,
    40592,
    40593,
    40594,
    40595,
    40596,
    40597,
)


@dataclass
class Split:
    X: np.ndarray
    y: np.ndarray


@dataclass
class Dataset:
    name: str
    id: int
    features: pd.DataFrame
    labels: pd.DataFrame
    openml: OpenMLDataset
    encoders: dict[str, LabelEncoder]

    def split(
        self,
        splits: Iterable[float],
        seed: int | None = 1,
    ) -> tuple[Split, ...]:
        """Create splits of the data

        Parameters
        ----------
        splits : Iterable[float]
            The percentages of splits to generate

        seed : int | None = None
            The seed to use for the splits

        Returns
        -------
        tuple[Split, ...]
            The collected splits
        """
        splits = list(splits)
        assert abs(1 - sum(splits)) <= 1e-6, "Splits must sum to 1"

        sample_sizes = tuple(int(s * len(self.features)) for s in splits)

        collected_splits = []

        next_xs = self.features.to_numpy()
        next_ys = self.labels.to_numpy()

        for size in sample_sizes[:-1]:
            xs, next_xs, ys, next_ys = train_test_split(
                next_xs, next_ys, train_size=size, random_state=seed
            )
            collected_splits.append(Split(X=xs, y=ys))
        collected_splits.append(Split(X=next_xs, y=next_ys))

        return tuple(collected_splits)

    @staticmethod
    def from_openml(id: int) -> Dataset:
        """Processes an multilabel OpenMLDataset into its features and targets

        Parameters
        ----------
        id: int
            The id of the dataset

        Returns
        -------
        Dataset
        """
        dataset = openml.datasets.get_dataset(id)
        print(dataset.name, id)
        targets = dataset.default_target_attribute.split(",")
        data, _, _, _ = dataset.get_data()

        assert isinstance(data, pd.DataFrame)

        # Process the features and turn all categorical columns into ints
        features = data.drop(columns=targets)
        encoders: dict[str, LabelEncoder] = {}

        for name, col in features.iteritems():
            if col.dtype in ["object", "category", "string"]:
                encoder = LabelEncoder()
                features[name] = encoder.fit_transform(col)
                encoders[name] = encoder

        labels = data[targets]

        # Since we assume binary multilabel data, we convert the labels
        # to all be boolean types
        labels = labels.astype(bool)

        return Dataset(
            name=dataset.name,
            id=id,
            features=features,
            labels=labels,
            openml=dataset,
            encoders=encoders,
        )

from sklearn.metrics import f1_score as sklearn_f1_score
import numpy as np

def f1_score(y_true: np.ndarray, y_pred:np.ndarray) -> float:
    return sklearn_f1_score(y_true, y_pred, average="macro", zero_division=0)

In [ ]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# get the dataset
def get_dataset():
	X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)
	return X, y

# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(20, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = f1_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results



In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from torch.functional import split
class MyDataset(Dataset):
 
  def __init__(self,split):
 
    x=np.array(split.X)
    y=np.array(split.y)
 
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [ ]:
def train_(dataloader, model, loss_fn, optimizer,logger,dataset_test=None,ftt=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X,None) if ftt else model(X) 
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
          yhat = torch.sigmoid(pred).round()
          f1_train = f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
          if(dataset_test is not None):
            f1_test = test(dataset_test,model,loss_fn,ftt)
            logger.log({"loss":loss,"F1_train":f1_train,"F1_val":f1_test})
          else:
            logger.log({"loss":loss,"F1_train":f1_train})
        del X,y,pred,loss

def train(dataloader, model, loss_fn, optimizer,dataset_test=None,ftt=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X,None) if ftt else model(X) 
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       

def test(dataloader, model, loss_fn,ftt=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    #print("num batches: "+str(num_batches))
    #print("size: "+str(size))
    model.eval()
    f1 = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X,None) if ftt else  model(X ) 
            pred = torch.sigmoid(pred)
            yhat = pred.round()
            # calculate accuracy
            f1 += f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
    f1 /= num_batches
    return f1
    

In [ ]:
def get_best_thresholds(dataloader,model,ftt=False):
  model.eval()
  with torch.no_grad():
      for X, y in dataloader:
          X, y = X.to(device), y.to(device)
          pred = model(X,None) if ftt else  model(X)
          pred = torch.sigmoid(pred)
  final_thresholds = np.array([])
  thresholds = np.array([.20,.40,.50,.60,80])
  for i in range(y.shape[1]):
    pred_ =pred[:,i]
    y_ = y[:,i]
    scores = np.array([])
    for threshold in thresholds:
      yhat = (pred_>threshold).float()
      score = f1_score(yhat.detach().cpu().numpy(),y_.detach().cpu().numpy())
      scores = np.append(scores,score)
    index = np.argmax(scores)
    final_thresholds = np.append(final_thresholds,thresholds[index])
  return final_thresholds

In [ ]:
def test_with_thresholds(dataloader, model, loss_fn,thresholds,ftt=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    f1 = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X,None) if ftt else  model(X) 
            pred = torch.sigmoid(pred)
            yhat = None
            for i in range(len(thresholds)):
              pred_ =pred[:,i]
              yhat_ = (pred_>thresholds[i]).float()
              if yhat== None:
                yhat = yhat_ 
              else: 
                yhat = torch.column_stack((yhat,yhat_))
            f1 = f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
    f1 /= num_batches
    return f1


In [ ]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 17071734784
free     : 17069506560
used     : 2228224


In [ ]:
import rtdl
x = torch.randn(4, 2)
config = { "d_in":x.shape[1],
    "n_blocks":5,
    "d_main":512,
    "d_hidden":512,
    "dropout_first":0.35,
    "dropout_second":0.0,
    "d_out":1}
module = rtdl.ResNet.make_baseline(**config)
assert module(x).shape == (len(x), 1)

print(module)

ResNet(
  (first_layer): Linear(in_features=2, out_features=512, bias=True)
  (blocks): Sequential(
    (0): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=512, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.35, inplace=False)
      (linear_second): Linear(in_features=512, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (1): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=512, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.35, inplace=False)
      (linear_second): Linear(in_features=512, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (2): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=

#Data Preprocesing 

In [ ]:
import openml
import numpy as np
import math
import time
from sklearn import preprocessing

def dataset_num_to_id(dataset_number):
    dataset_ids = [40590, 40591, 40592, 40593, 40594, 40595, 40596, 40597, 40588, 40589]
    return dataset_ids[dataset_number]


class OpenMlDataset():
    def __init__(self, dataset_id, threshold = 0.8, normalization_type="standard", split=0.8, seed=0) -> None:
        self.dataset_id = dataset_id
        self.random_generator = np.random.RandomState(seed)
        self.split = split
        self.load_dataset()
        self.pred_preprocessing = PredictorsPreprocessingModule(self.pred_data, self.pred_categorical_indicator, correlation_threshold=threshold, normalization_type=normalization_type)
        #self.target_preprocessing = TargetsPreprocessingModule(self.target_data, transformation_name=transformation)
        self.pred_data = self.pred_preprocessing.get_data()
        self.train_test_split()
        #self.target_data = self.target_preprocessing.get_data()
    
    def load_dataset(self):
        dataset = openml.datasets.get_dataset(self.dataset_id)
        self.targets_name = dataset.default_target_attribute.split(",")
        
        X, _, categorical_indicator, features_name = dataset.get_data(dataset_format="dataframe")
        self.features_name = features_name
        self.categorical_indicator = categorical_indicator
        
        for i, name in enumerate(self.features_name):
            if not self.categorical_indicator[i]:
                continue
            encoder = preprocessing.LabelEncoder()
            encoder.fit(X[name])
            column = encoder.transform(X[name])
            X.loc[:, name] = column.astype('float32')
        
        X = X.astype('float32')
        self.data = X.to_numpy()
        
        print("Chosen dataset is ",dataset.name, " (id:"+str(self.dataset_id)+")")
        print("Original dataset shape is ",self.data.shape)
        print("Number of predictors : ", len(self.features_name) - len(self.targets_name), "Number of targets : ", len(self.targets_name))

        # Split data into predictors and targets
        target_inds = []
        pred_inds = []
        for i in range(self.data.shape[1]):
            if self.features_name[i] in self.targets_name:
                target_inds.append(i)
            else:
                pred_inds.append(i)
        self.target_categorical_indicator = [self.categorical_indicator[i] for i in target_inds]
        self.pred_categorical_indicator = [self.categorical_indicator[i] for i in pred_inds]
        self.pred_data = self.data[:, pred_inds]
        self.target_data = self.data[:, target_inds]


    def get_predictors(self):
        return self.train_pred_data, self.pred_categorical_indicator
        
    
    def get_targets(self):
        return self.train_tar_data, self.target_categorical_indicator
        
        
    def get_test_predictors(self):
        return self.test_pred_data, self.pred_categorical_indicator
        
    
    def get_test_targets(self):
        return self.test_tar_data, self.target_categorical_indicator

    
    def get_dataset(self):
        return self.pred_data, self.target_data
    

    def train_test_split(self):
        #print(type(self.pred_data))
        #print(type(self.target_data))
        perm_idxs = np.arange(self.data.shape[0])
        self.random_generator.shuffle(perm_idxs)
        split_idx = int(self.data.shape[0]*self.split)  
        train_idxs = perm_idxs[:split_idx]
        valid_idxs = perm_idxs[split_idx:]
        self.train_pred_data = self.pred_data[train_idxs]
        self.train_tar_data = self.target_data[train_idxs]
        self.test_pred_data = self.pred_data[valid_idxs]
        self.test_tar_data = self.target_data[valid_idxs] 



class PredictorsPreprocessingModule():
    def __init__(self, data, categorical_indicator, correlation_threshold = 0.8, normalization_type = "standard") -> None:
        self.data = data
        self.categorical_indicator = categorical_indicator
        self.correlation_threshold = correlation_threshold
        self.normalization_type = normalization_type
        assert self.data.shape[1] == len(self.categorical_indicator), "Categorical indicator should have length equal to the number of features"
        self.features_to_remove_list = []
        self.scaler_list = []
        self.preprocessing()
    
    def preprocessing(self):
        self.transform_2_value_numeric_to_binary()
        self.check_for_single_value_parameters()
        #self.check_for_duplicated_samples()
        self.check_for_values_with_one_occurance()
        self.check_for_pseudocorrelation()
        self.check_for_correlation()
        # Must be the last
        self.normalize_dataset()
        self.dataset_one_hot_encoding()
    
    
    def transform_2_value_numeric_to_binary(self):
        transformed_features_num = 0
        for i in range(self.data.shape[1]):
            if self.categorical_indicator[i]:
                pass
            unique_values = np.unique(self.data[:, i])
            if unique_values.shape[0] == 2:
                encoder = preprocessing.LabelEncoder()
                encoder.fit(self.data[:, i])
                
                self.data[:, i] = (encoder.transform(self.data[:, i])).astype('float32')
                self.categorical_indicator[i] = True
                transformed_features_num = transformed_features_num + 1

        print(transformed_features_num, " numeric features were transformed into binary")
        
        
    def check_for_duplicated_samples(self):
        
        old = self.data.shape[0]
        self.data = np.unique(self.data, axis=0)
        new = self.data.shape[0]
        print("Removed ",old - new," samples due to being duplicates")
        
       
    def check_for_values_with_one_occurance(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            if not self.categorical_indicator[i]:
                continue
            temp = (self.data[:,i]).astype(int)
            temp = temp + abs(int(np.amin(temp)))+1
            occurances = np.bincount(temp)
            if 1 in occurances:
                params_to_delete.append(i)

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with 1 occurance of value")
        #print(params_to_delete)
        
        
    def check_for_single_value_parameters(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            unique_values = np.unique(self.data[:,i])
            if unique_values.shape[0] == 1:
                params_to_delete.append(i)

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with one unique value")
        #print(params_to_delete)
    
    def check_for_pseudocorrelation(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            if not self.categorical_indicator[i]:
                continue
            for j in np.arange(i-1, -1, -1):
                if not self.categorical_indicator[j]:
                    continue
                # Check for pseudocorrelation
                temp_i = self.data[:,i]
                temp_j = self.data[:,j]
                uniq_val_num_i = np.unique(self.data[:,i]).shape[0]
                uniq_val_num_j = np.unique(self.data[:,j]).shape[0]
                if uniq_val_num_i > uniq_val_num_j:
                    lower = j
                else:
                    lower = i
                temp_i = temp_i * uniq_val_num_j
                sum = temp_i + temp_j
                uniq_val_sum = np.unique(sum)
                if uniq_val_sum.shape[0] == uniq_val_num_i or uniq_val_sum.shape[0] == uniq_val_num_j:
                    if not(lower in params_to_delete):
                        params_to_delete.append(lower)

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with pseudocorrelated values")
              
              
    def check_for_correlation(self):
        params_to_delete = []
        for i in range(self.data.shape[1]):
            if self.categorical_indicator[i]:
                continue
            for j in range(i+1, self.data.shape[1]):
                if self.categorical_indicator[j]:
                    continue
                
                sum_i = np.sum(self.data[:, i])
                sum_j = np.sum(self.data[:, j])
                sum_i2 = np.sum(self.data[:, i]**2)
                sum_j2 = np.sum(self.data[:, j]**2)
                sum_ij = np.sum(self.data[:, i] * self.data[:, j])
                n = self.data.shape[0]
                
                correlation = (n*sum_ij - sum_i * sum_j)/(math.sqrt((n * sum_i2 - (sum_i)**2)*(n * sum_j2 - (sum_j)**2)))
                correlation = abs(correlation)
                if correlation>= self.correlation_threshold:
                    #print("correlation ",correlation)
                    #print(" i ",i," j ",j)
                    if np.unique(self.data[:, i]).shape[0] >= np.unique(self.data[:, j]).shape[0]:
                        if not(j in params_to_delete):
                            params_to_delete.append(j)
                    else:
                        if not(i in params_to_delete):
                            params_to_delete.append(i)
                
        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with correlated values")
        #print(params_to_delete)
            
                 
    def clear_feature_data(self, indices:list):
        indices.sort()
        self.features_to_remove_list.append(indices)
        indices = indices[::-1]
        for i in indices:
            self.categorical_indicator.pop(i)
       
       
    def normalize_dataset(self):
        for i in range(self.data.shape[1]):
            if self.categorical_indicator:
                self.scaler_list.append(None)
                continue
            
            if self.normalization_type == "standard":
                self.scaler_list.append(preprocessing.StandardScaler())
                self.scaler_list[i].fit(self.data[:, i].reshape(-1, 1))
                self.data[:, i] = self.scaler_list[i].transform(self.data[:, i].reshape(-1, 1)).reshape((-1))
            elif self.normalization_type == "robust":
                self.scaler_list.append(preprocessing.RobustScaler())
                self.scaler_list[i].fit(self.data[:, i].reshape(-1, 1))
                self.data[:, i] = self.scaler_list[i].transform(self.data[:, i].reshape(-1, 1)).reshape((-1))
            else:
                modes = ["standard", "robust"]
                print("Wrong normalization mode. Choose one of the following: ", modes)
                
                
    def one_hot_encoding(self, x):
        length = x.shape[0]
        max_value = int(np.max(x))
        new_array = np.zeros((length, max_value+1))
        new_array[np.arange(length), x.astype(int)] = 1.0
        return new_array 
               
                
    def dataset_one_hot_encoding(self):
        for i in range(self.data.shape[1]-1, -1, -1):
            if np.unique(self.data[:, i]).shape[0] == 2 or not self.categorical_indicator[i]:
                temp_ar = np.reshape(self.data[:, i], (-1, 1))
            else:
                temp_ar = self.one_hot_encoding(self.data[:,i])
            
            if i+1 == self.data.shape[1]:
                self.data = np.concatenate((self.data[:,:i], temp_ar), axis=1)
            else:
                self.data = np.concatenate((self.data[:,:i], temp_ar, self.data[:,i+1:]), axis=1)
    
                
    def get_data(self):
        return self.data
            

    # def transform_validation(self, valid_data):
    #     for i in range(len(self.features_to_remove_list)):
    #         valid_data = np.delete(valid_data, self.features_to_remove_list[i], axis = 1)
    #     for i in range(len(self.scaler_list)):
    #         if self.scaler_list[i] is None:
    #             continue
    #         valid_data[:,i] = self.scaler_list[i].transform(valid_data[:, i].reshape(-1, 1)).reshape((-1))
    #     return valid_data
        

class TargetsPreprocessingModule():
    def __init__(self, data, transformation_name = "pt3") -> None:
        self.data = data
        self.transformation_name = transformation_name
        self.choose_transformation()
        self.dataset_one_hot_encoding()
    
    
    def choose_transformation(self):
        if self.transformation_name == "pt3":
            self.pt3()
        else:
            assert False, "Invalid transformation name"        
            
    def pt3(self):
        self.unique_vectors = np.unique(self.data, axis=0)
        print("Found ", self.unique_vectors.shape[0], "unique target vectors")
        indices_list = []
        
        for i in range(self.unique_vectors.shape[0]):
            temp = abs(self.data - self.unique_vectors[i])
            sum = np.sum(temp, axis=1)
            indices = np.nonzero(sum == 0)[0]
            self.data[indices] = i
        
        self.data = self.data[:,0].reshape(-1,1)
    
    # def pt3_val(self, val_data):
    #     for i in range(self.unique_vectors.shape[0]):
    #         temp = abs(val_data - self.unique_vectors[i])
    #         sum = np.sum(temp, axis=1)
    #         indices = np.nonzero(sum == 0)[0]
    #         if indices.size == 0:
    #             continue
    #         val_data[indices] = i
        
    #     val_data = val_data[:,0].reshape(-1,1)
    #     val_data = self.dataset_one_hot_encoding(val_data)
    #     return val_data
                
    
    def one_hot_encoding(self, x):
        length = x.shape[0]
        max_value = int(np.max(x))
        new_array = np.zeros((length, max_value+1))
        new_array[np.arange(length), x.astype(int)] = 1.0
        return new_array 
               
                
    def dataset_one_hot_encoding(self):
        for i in range(self.data.shape[1]-1, -1, -1):
            if np.unique(self.data[:, i]).shape[0] == 2:
                temp_ar = np.reshape(self.data[:, i], (-1, 1))
            else:
                temp_ar = self.one_hot_encoding(self.data[:,i])
            
            if i+1 == self.data.shape[1]:
                self.data = np.concatenate((self.data[:,:i], temp_ar), axis=1)
            else:
                self.data = np.concatenate((self.data[:,:i], temp_ar, self.data[:,i+1:]), axis=1)
        return self.data
    
    def get_data(self):
        return self.data
    
        

class CrossValidation():
    def __init__(self, pred_data, target_data, split_num) -> None:
        self.pred_data = pred_data
        self.target_data = target_data
        self.split_num = split_num
        self.make_splits()
        self.iter = 0    
    
    def make_splits(self):
        samples_per_split = self.pred_data.shape[0]//self.split_num + 1
        self.pred_data_per_split = []
        self.target_data_per_split = []
        for i in range(self.split_num):
            self.pred_data_per_split.append(self.pred_data[i*samples_per_split : min((i+1)*samples_per_split, self.pred_data.shape[0])])
            self.target_data_per_split.append(self.target_data[i*samples_per_split : min((i+1)*samples_per_split, self.target_data.shape[0])])

    
    def __iter__(self):
        return self
    
    
    def __next__(self):
        if self.iter == self.split_num:
            self.iter = 0
            raise StopIteration
        
        valid_pred = self.pred_data_per_split[self.iter]
        valid_target = self.target_data_per_split[self.iter]
        train_pred = None
        train_target = None
        for i in range(self.split_num):
            if i == self.iter:
                continue
            if train_pred is None:
                train_pred = self.pred_data_per_split[i]
                train_target = self.target_data_per_split[i]
            else:
                train_pred = np.concatenate((train_pred, self.pred_data_per_split[i]), axis=0)
                train_target = np.concatenate((train_target, self.target_data_per_split[i]), axis=0)
        self.iter = self.iter + 1
        return train_pred, train_target, valid_pred, valid_target
    
    
    def __getitem__(self, index):
        valid_pred = self.pred_data_per_split[index]
        valid_target = self.target_data_per_split[index]
        train_pred = None
        train_target = None
        for i in range(self.split_num):
            if i == index:
                continue
            if train_pred is None:
                train_pred = self.pred_data_per_split[i]
                train_target = self.target_data_per_split[i]
            else:
                train_pred = np.concatenate((train_pred, self.pred_data_per_split[i]), axis=0)
                train_target = np.concatenate((train_target, self.target_data_per_split[i]), axis=0)
        return train_pred, train_target, valid_pred, valid_target
    
    
    def __len__(self):
        return self.split_num


class Split:
  def __init__(self, x, y):
    self.X = x
    self.y = y

# dataset_names = ["birds", "emotions", "enron", "genbase",
#                          "image", "langLog", "reuters", "scene",
#                          "slashdot", "yeast"]
# for idx, i in enumerate(task_ids):
#     dataset = OpenMlDataset(i)
#     print(dataset_names[idx])
#     pred_data, _ = dataset.get_predictors()
#     target_data, _ = dataset.get_targets()
    
#     cross_validation = CrossValidation(pred_data=pred_data, target_data=target_data, split_num=3)
    
#     print(len(cross_validation))
    

#     score_tests = np.array([])

#     for i, (tp, tt, vp, vt) in enumerate(cross_validation):
#         loss_fn = nn.BCEWithLogitsLoss()


#         train_split = Split(x=tp,y=tt)
#         validation_split = Split(x=vp,y=vt)

        
#         train_dataset = MyDataset(train_split)
#         test_dataset = MyDataset(validation_split)

        
#         train_dataloader = DataLoader(train_dataset, batch_size=256)
#         test_dataloader = DataLoader(test_dataset, batch_size=256)

#         n_inputs, n_outputs = tp.shape[1], tt.shape[1]


#         name = "ResNet_DEHB_" + dataset_names[i]
#         config = { "d_in":n_inputs,
#           "n_blocks":5,
#           "d_main":512,
#           "d_hidden":256,
#           "dropout_first":.3,
#           "dropout_second":.2,
#           "d_out":n_outputs}


#         model = rtdl.ResNet.make_baseline(**config).to(device)  

#         optimizer = torch.optim.AdamW(model.parameters(), lr=.0003, weight_decay=.5)
#         # print(i, " ", tp.shape, " ", tt.shape, " ", vp.shape, " ", vt.shape,)
#         # print(np.sum(tp, axis=(0,1)), " ", np.sum(tt, axis=(0,1)), " ", np.sum(vp, axis=(0,1)), " ", np.sum(vt, axis=(0,1)), " ",  )
        
        
#         budget = 250
#         for t in range(budget):
#           train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
#         score_test = test(test_dataloader,model, loss_fn,ftt=False)

#         score_tests = np.append(score_tests,score_test)
#         print("F1 score: " + str(score_test))
#     print(np.mean(score_tests))
#     print(np.std(score_tests))




    # j = 1
    # tp, tt, vp, vt = cross_validation[j]
    # print(j)
    # print(np.sum(tp, axis=(0,1)), " ", np.sum(tt, axis=(0,1)), " ", np.sum(vp, axis=(0,1)), " ", np.sum(vt, axis=(0,1)), " ",  )
    
    # for i, (tp, tt, vp, vt) in enumerate(cross_validation):
    #     print(i, " ", tp.shape, " ", tt.shape, " ", vp.shape, " ", vt.shape,)
    #     print(np.sum(tp, axis=(0,1)), " ", np.sum(tt, axis=(0,1)), " ", np.sum(vp, axis=(0,1)), " ", np.sum(vt, axis=(0,1)), " ",  )

#DE

In [ ]:
import matplotlib.pyplot as plt
import rtdl
import wandb

dataset_names = ["birds", "emotions", "enron", "genbase",
                         "image", "langLog", "reuters", "scene",
                         "slashdot", "yeast"]

In [ ]:
names = {    40588:"birds",
    40589:"emotions",
    40590:"enron",
    40591:"genbase",
    40592:"image",
    40593:"langLog",
    40594:"reuters",
    40595:"scene",
    40596:"slashdot",
    40597:"yeast"}

names[40595]

'scene'

In [ ]:

def obj(x,budget,**kwargs):

    seed = 42
    dataset = kwargs['dataset']
    
    cross_validation = CrossValidation(pred_data=dataset.train_pred_data, target_data=dataset.train_tar_data, split_num=3)

    score_tests = np.array([])

    for i, (tp, tt, vp, vt) in enumerate(cross_validation):
        loss_fn = nn.BCEWithLogitsLoss()


        train_split = Split(x=tp,y=tt)
        validation_split = Split(x=vp,y=vt)

        
        train_dataset = MyDataset(train_split)
        test_dataset = MyDataset(validation_split)

        
        train_dataloader = DataLoader(train_dataset, batch_size=256)
        test_dataloader = DataLoader(test_dataset, batch_size=256)

        n_inputs, n_outputs = tp.shape[1], tt.shape[1]

        config = { "d_in":n_inputs,
        "n_blocks":x["n_blocks"],
        "d_main":x["d_main"],
        "d_hidden":x["d_hidden"],
        "dropout_first":x["dropout_first"],
        "dropout_second":x["dropout_second"],
        "d_out":n_outputs}

        model = rtdl.ResNet.make_baseline(**config).to(device)  

        config["batch_size"] = 256
        config["epoch"] = budget
        config["weight_decay"] = x['weight_decay']

        optimizer = torch.optim.AdamW(model.parameters(), lr=x["lr"], weight_decay=x['weight_decay'])
        #Only save the first run
        if i==0:
          with wandb.init(
                  project="multilabel",
                  config=config,
                  group=str(dataset.dataset_id),
                  reinit=True,
                  mode="offline",
                  settings=wandb.Settings(start_method="thread")):

              for t in range(budget):
                  train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
                  score_test = test(test_dataloader,model, loss_fn,ftt=False)

              score_tests = np.append(score_tests,score_test)
        else:
            for t in range(budget):
                  train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
                  score_test = test(test_dataloader,model, loss_fn,ftt=False)

            score_tests = np.append(score_tests,score_test)

    print(config)
    print(np.mean(score_tests))
    return {'mean_F1':np.mean(score_tests), 'std_F1':np.std(score_tests)}


In [ ]:

def obj_withno_preprocesing(x,budget,dataset_id):

  seed = 42

  loss_fn = nn.BCEWithLogitsLoss()

  dataset = Dataset.from_openml(dataset_id)
  train_split, test_split = dataset.split(splits=(0.75, 0.25), seed=seed)
  n_inputs, n_outputs = len(dataset.features.columns), len(dataset.labels.columns)

  name = "ResNet_DEHB_" + dataset.name
  config = { "d_in":n_inputs,
    "n_blocks":x["n_blocks"],
    "d_main":x["d_main"],
    "d_hidden":x["d_hidden"],
    "dropout_first":x["dropout_first"],
    "dropout_second":x["dropout_second"],
    "d_out":n_outputs}


  model = rtdl.ResNet.make_baseline(**config).to(device)  

  config["batch_size"] = x["batch_size"]
  config["epoch"] = budget
  config["weight_decay"] = x['weight_decay']

  print(config)

  optimizer = torch.optim.AdamW(model.parameters(), lr=x["lr"], weight_decay=x['weight_decay'])

  train_dataset = MyDataset(train_split)
  test_dataset = MyDataset(test_split)

  train_dataloader = DataLoader(train_dataset, batch_size=512)
  test_dataloader = DataLoader(test_dataset, batch_size=512)

  for t in range(budget):
    train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
  
  score_test = test(test_dataloader,model, loss_fn,ftt=False)

  del model
  torch.cuda.empty_cache()

  print("f1_score: " + str(score_test))
  
  return {"f1_score": score_test}

In [ ]:
import json
import os
import time
from typing import Callable, Union

import ConfigSpace
import numpy as np
from ConfigSpace.util import (deactivate_inactive_hyperparameters,
                              impute_inactive_values)

SEED = 42


class ConfigVectorSpace(ConfigSpace.ConfigurationSpace):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.hyperparameters = self.get_hyperparameters()
        self.dim = len(self.hyperparameters)

        self.name_to_idx = {}
        for i, hp in enumerate(self.hyperparameters):
            # maps hyperparameter name to positional index in vector form
            self.name_to_idx[hp.name] = i
        
    def sample_vectors(self, size):
        configurations = super().sample_configuration(size)
        if not isinstance(configurations , list):
            configurations = [configurations]

        vectors = [self._to_vector(config) for config in configurations]
        return vectors
    
    def _to_vector(self, config: ConfigSpace.Configuration) -> np.array:
        '''Converts ConfigSpace.Configuration object to numpy array scaled to [0,1]
        Works when self is a ConfigVectorSpace object and the input config is a ConfigSpace.Configuration object.
        Handles conditional spaces implicitly by replacing illegal parameters with default values
        to maintain the dimensionality of the vector.
        '''        
        config = impute_inactive_values(config)

        vector = [None] * self.dim
        self.name_to_id = dict()

        for name in config:
            idx = self.name_to_idx[name]
            hyper = self[name]
            if type(hyper) == ConfigSpace.OrdinalHyperparameter:
                nlevels = len(hyper.sequence)
                vector[idx] = hyper.sequence.index(config[name]) / nlevels
            elif type(hyper) == ConfigSpace.CategoricalHyperparameter:
                nlevels = len(hyper.choices)
                vector[idx] = hyper.choices.index(config[name]) / nlevels
            else:
                bounds = (hyper.lower, hyper.upper)
                param_value = config[name]
                if hyper.log:
                    vector[idx] = np.log(param_value / bounds[0]) / np.log(bounds[1] / bounds[0])
                else:
                    vector[idx] = (config[name] - bounds[0]) / (bounds[1] - bounds[0])
        return np.array(vector)
    
    def to_config(self, vector: np.array) -> ConfigSpace.Configuration:
        '''Converts numpy array to ConfigSpace.Configuration object
        Works when self is a ConfigVectorSpace object and the input vector is in the domain [0, 1].
        '''
        # creates a ConfigSpace object dict with all hyperparameters present, the inactive too
        new_config = impute_inactive_values(self.sample_configuration()).get_dictionary()

        # iterates over all hyperparameters and normalizes each based on its type
        for hyper in self.hyperparameters:
            idx = self.name_to_idx[hyper.name]

            if type(hyper) == ConfigSpace.OrdinalHyperparameter:
                ranges = np.arange(start=0, stop=1, step=1/len(hyper.sequence))
                param_value = hyper.sequence[np.where((vector[idx] < ranges) == False)[0][-1]]
            elif type(hyper) == ConfigSpace.CategoricalHyperparameter:
                ranges = np.arange(start=0, stop=1, step=1/len(hyper.choices))
                param_value = hyper.choices[np.where((vector[idx] < ranges) == False)[0][-1]]
            else:  # handles UniformFloatHyperparameter & UniformIntegerHyperparameter
                # rescaling continuous values
                if hyper.log:
                    log_range = np.log(hyper.upper) - np.log(hyper.lower)
                    param_value = np.exp(np.log(hyper.lower) + vector[idx] * log_range)
                else:
                    param_value = hyper.lower + (hyper.upper - hyper.lower) * vector[idx]
                if type(hyper) == ConfigSpace.UniformIntegerHyperparameter:
                    param_value = int(np.round(param_value))  # converting to discrete (int)
                else:
                    param_value = float(param_value)
            new_config[hyper.name] = param_value

        #self.check_configuration(ConfigSpace.Configuration(self, values=new_config))
        #new_config = impute_inactive_values(ConfigSpace.Configuration(self, values=new_config))
        # the mapping from unit hypercube to the actual config space may lead to illegal
        # configurations based on conditions defined, which need to be deactivated/removed
        new_config = deactivate_inactive_hyperparameters(
            configuration = new_config, configuration_space=self
        )
        return new_config


class DE(object):
    def __init__(self,
        space : ConfigVectorSpace,
        crossover_prob : float = 0.9,
        mutation_factor : float = 0.8,
        metric = "f1_Score",
        mode = "max",
        rs: np.random.RandomState=None,
        bound_control = "random",
        save_path : Union[str, None] ="./data.json",
        save_freq=10) -> None:

        assert 0 <= crossover_prob <= 1, ValueError("crossover_prob given is not a probability")
        assert 0 <= mutation_factor <= 2, ValueError("mutation_factor not in range [0, 2]")
        assert mode in ["min", "max"], ValueError("Valid optimization mode in ['min', 'max']")
        
        self.space = space
        self.crossover_prob = crossover_prob
        self.mutation_factor = mutation_factor
        self.metric = metric
        self.mode = mode
        self.rs = rs
        self.bound_control = bound_control
        self.save_path = save_path
        self.save_freq = save_freq
        
        self.traj = []
        self.runtime = []
        self.inc_config = None
        self.inc_score = float("inf") if self.mode == "min" else float("-inf")

        self.histroy = []

        self._min_pop_size = 3
        self._eval_counter = 0
        self._iteration_counter = -1

    def _init_population(self, pop_size : int) -> np.ndarray :

        # sample from ConfigSpace s.t. conditional constraints (if any) are maintained
        population = self.space.sample_vectors(size=pop_size)
        return np.asarray(population)
    
    def _eval_population(self, obj : Callable, population: Union[np.ndarray, list] , budget, **kwargs) -> np.ndarray:
        fitness = []

        for candidate in population:
            if self._is_termination():
                raise StopIteration

            config = self.space.to_config(candidate)
            result = obj(config, budget, **kwargs)

            assert isinstance(result, dict), TypeError("Objective function must return a dictionary")
            assert self.metric in result, KeyError(f"Given mteric '{self.metric}' not found in dictionary {result}, returned by the objective function")
            score = result[self.metric]

            condition = {
                "min" : score < self.inc_score,
                "max" : score > self.inc_score, 
            } 

            config_dict = config.get_dictionary()
            config_dict.update({"budget": budget})

            if condition[self.mode]:
                self.inc_config = config_dict
                self.inc_score = score

            fitness.append(score)
            # trajectory updated at every fn eval, regardless of save_freq
            self.traj.append(self.inc_score)
            self.runtime.append(self._time_elapsed())
            self._update_history(candidate, result, budget)

            self._eval_counter += 1

            # Update history and wrtie data every 'save_freq' objective fn evaluations
            if self._eval_counter % self.save_freq == 0:
                self.save_data()

        return np.asarray(fitness)

    def _update_history(self, candidate, result, budget):
        record = {
            "candidate": candidate.tolist(),
            "budget": budget}
        
        record.update(result)
        self.histroy.append(record)

    def _sample(self, population, size, replace=False):
        selection = self.rs.choice(np.arange(len(population)), size, replace=replace)
        return population[selection]

    def _mutation(self, population: np.ndarray):
        pop_size = len(population)
        assert pop_size > self._min_pop_size, ValueError(f"Population too small ( < DE()._min_pop_size = {self._min_pop_size}) for mutation")

        base, a, b = self._sample(population, self._min_pop_size)

        diff = a - b
        mutant = base + self.mutation_factor * diff
        return mutant

    def _crossover(self, candidate: np.ndarray, mutant: np.ndarray) -> np.ndarray:
        # Sample a random value from U[0, 1] for every dimension
        p = self.rs.rand(self.space.dim)

        # perform binomial crossover
        child = np.asarray([mutant[i] if p[i] < self.crossover_prob else candidate[i] for i in range(self.space.dim)])
        return child
    
    def _selection(self, population: np.ndarray, children: np.ndarray, fitness: np.ndarray, children_fitness: np.ndarray):
        # Conduct parent-child competition and select new population 
        pop_size = len(population)
        for i in range(pop_size):
            condition = {
                "max" : children_fitness[i] >= fitness[i],
                "min" : children_fitness[i] <= fitness[i]
                }
            if condition[self.mode]:
                population[i] = children[i]
                fitness[i] = children_fitness[i]
        
        return population, fitness
    
    def _check_bounds(self, vector: np.ndarray) -> np.ndarray:

        violations = np.where((vector > 1) | (vector < 0))[0]
        if len(violations) == 0:
            return vector
        if self.bound_control == 'random':
            vector[violations] = self.rs.uniform(low=0.0, high=1.0, size=len(violations))
        else:
            # Can be exploited by optimizer if solution at clip limits 
            vector[violations] = np.clip(vector[violations], a_min=0, a_max=1)
        return vector

    def _next_generation(self, population, alt_pop=None):
        children = []

        for candidate in population:
            # If alt_pop is None, vanilla mutation is perfomed
            mutant = self._mutation(alt_pop if alt_pop is not None else population)
            mutant = self._check_bounds(mutant)
            child = self._crossover(candidate, mutant)
            children.append(child)
        
        return children

    def optimize(self, obj : Callable, budget=None, pop_size : Union[int,None] = None, limit: int = 10, unit : str = "iter", **kwargs):
        self._set_limit(limit, unit)

        if pop_size is None:
            pop_size = 10 * self.space.dim # heuristic

        try: 
            # Initialize and Evaluate the population
            population = self._init_population(pop_size)
            fitness = self._eval_population(obj, population, budget, **kwargs)

            # Until budget is exhausted
            while not self._is_termination(limit, unit):
                self._iteration_counter += 1

                children = self._next_generation(population)

                children_fitness = self._eval_population(obj, children, budget)
                population, fitness = self._selection(population, children, fitness, children_fitness)
        
        except StopIteration:
            return self.inc_config
    
    def _set_limit(self, limit : int, unit : str):
        self.limit = limit
        self.unit = unit
        self._start_timer()
     
    def _start_timer(self):
        self._wall_clock_start = time.time()
    
    def _time_elapsed(self):
        # time difference in sec
        diff = time.time() - self._wall_clock_start
        return diff

    def _is_termination(self):
        assert self.unit in ["hr", "min", "sec", "evals", "iter"], ValueError("Unrecognized unit given")

        if self.unit in ["hr", "min", "sec"]:
            scale = {
                "sec" : 1,
                "min" : 60,
                "hr"  : 60 * 60
            }
            diff = self._time_elapsed()
            return diff >= self.limit * scale[self.unit]
        elif self.unit == "evals":
            return self._eval_counter >= self.limit
        else:
            return self._iteration_counter >= self.limit

    def save_data(self):
        data = {
            "params" : self._init_params(),
            "result" : {
                "best_config": self.inc_config,
                "best_score" : self.inc_score,
            },
            "traj": self.traj,
            "runtime": self.runtime,
            "history" : self.histroy,
        }

        if self.save_path is not None:
            # path = os.path.join(self.save_path, "data.json")
            # os.makedirs(save_path, exist_ok=True)
            os.makedirs(os.path.dirname(self.save_path), exist_ok=True)
            with open(self.save_path, "w+") as outfile:
                json.dump(data, outfile)
    
    def _init_params(self):
        params = {
            "crossover_prob" : self.crossover_prob,
            "mutation_factor" : self.mutation_factor,
            "metric" : self.metric,
            "mode" : self.mode,
            "seed" : SEED,
            "bound_control" : self.bound_control,
            "iters" : self._iteration_counter,
            "evals" : self._eval_counter

        }
        return params

class DEHB(DE):
    def __init__(self, 
        space: ConfigVectorSpace,
        min_budget : int = 10,
        max_budget : int = 100,
        eta : int = 2,
        crossover_prob: float = 0.9, 
        mutation_factor: float = 0.8, 
        metric = "f1_score",
        mode = "max",
        rs: np.random.RandomState=None,
        bound_control = "random",
        save_path : Union[str, None] ="./data.json",
        save_freq=10) -> None:

        super().__init__(space=space,
            crossover_prob=crossover_prob, 
            mutation_factor=mutation_factor,
            metric=metric,
            mode=mode,
            rs=rs,
            bound_control=bound_control,
            save_path=save_path,
            save_freq=save_freq
            )
        
        self.min_budget = min_budget
        self.max_budget = max_budget

        self.eta = eta
        
        self._all_in_one = self._get_bracket()
        self._SH_iter = len(self._all_in_one)

        self._genus = None
    
    def _get_bracket(self):
        # max num of eliminations in a bracket 
        s_max = int(np.floor(np.log(self.max_budget / self.min_budget) / np.log(self.eta)))

        # num of downsampling left at stage i in range(s_max + 1)
        n_downsampling = np.linspace(start=s_max, stop=0, num=s_max + 1)

        budgets = (self.max_budget * np.power(self.eta, -n_downsampling)).tolist()
        n_configs = (np.power(self.eta, n_downsampling)).tolist()

        bracket = tuple((int(n), int(b)) for n, b in zip(n_configs, budgets))
        return bracket

    def _init_eval_genus(self, obj : Callable, **kwargs):
        genus = dict()

        for (pop_size, budget) in self._all_in_one:
            species = dict()
            species["population"] = self._init_population(pop_size)
            species["fitness"] = self._eval_population(obj, species["population"], budget, **kwargs)
            
            #genus[budget] = species
            genus[budget] = self._sort_species(species)
        return genus
    
    def _sort_species(self, species : np.ndarray):
        species = species.copy()
        ranking = np.argsort(species["fitness"])
        if self.mode == "max":
            ranking = ranking[::-1]
        
        species["population"] = species["population"][ranking]
        species["fitness"] = species["fitness"][ranking]

        return species

    def _select_promotions(self, target : dict, previous : dict):
        promotions = []
        pop_size = len(target["population"])

        for individual in previous["population"]:
            # If individual already in target, then ignore it to minimize fn_evals
            if np.any(np.all(individual == target["population"], axis=1)):
                continue
            
            promotions.append(individual)
        
        if len(promotions) >= pop_size:
            promotions = promotions[:pop_size]
        else:
            return previous["population"][:pop_size]
            # raise BufferError("Not enough to promote")
            # Can simply pick top pop_size many individuals even 
            # if duplicates exist in target
        
        return np.asarray(promotions)
    
    def _get_alt_population(self, target : dict, previous : dict):

        # stage == 0, previous is None
        if previous is None:
            
            # Edge case where stage==0, but target population too small
            # for vanilla mutation, so we need an alt_pop that is not None
            if len(target) < self._min_pop_size :
                previous = target
            else:
                return None

        pop_size = len(target["population"])
        alt_pop = previous["population"][:pop_size]

        if len(alt_pop) < self._min_pop_size:
            filler_size = self._min_pop_size - len(alt_pop) + 1
            filler_pop = self._sample(self.global_pupulation, filler_size)

            alt_pop = np.concatenate([filler_pop, alt_pop])
        
        return alt_pop
    
    def optimize(self, obj : Callable, limit : int = 10, unit : str = "iter", **kwargs):
        self._set_limit(limit, unit)

        try:
            self.genus = self._init_eval_genus(obj, **kwargs)

            while True: # DEHB iteration
                self._iteration_counter += 1

                for j in range(self._SH_iter): # SH iterations
                    
                    previous = None
                    bracket = self._all_in_one[j:]

                    for stage, (pop_size, budget) in enumerate(bracket): # stages in a bracket
                        target =  self.genus[budget]

                        # Only True for first DEHB iteration and non-inital SH stage
                        promotion = True if self._iteration_counter == 0 and stage > 0 else False
                        if promotion:
                            children = self._select_promotions(target, previous)
                        else:
                            alt_pop = self._get_alt_population(target, previous)
                            children = self._next_generation(target["population"], alt_pop)

                        children_fitness = self._eval_population(obj, children, budget, **kwargs)
                        target["population"], target["fitness"] = self._selection(target["population"], children, target["fitness"], children_fitness)

                        target = self._sort_species(target)
                        self.genus[budget] = target
                        previous = target

        except StopIteration :
            return self.inc_config
            
    def _init_params(self):
        params = {
            "min_budget" : self.min_budget,
            "max_budget" : self.max_budget,
            "eta"  : self.eta
        }
        params.update(super()._init_params())
        return params

    @property
    def global_pupulation(self):
        assert self.genus is not None, AssertionError("No populaiton in genus initialized")
        return np.concatenate([species["population"] for species in self.genus.values()])




if __name__ == "__main__":

    dataset_id =40588

    current_dataset = OpenMlDataset(dataset_id,split=0.75,seed=42)

    current_dataset.train_test_split()

    rs = np.random.RandomState(seed=SEED)
    
    space = ConfigVectorSpace(
        name="neuralnetwork",
        seed=SEED,
        space={
            "lr": ConfigSpace.UniformFloatHyperparameter("lr", lower=1e-4, upper=1e-3, log=True, default_value=1e-3),
            "dropout_first": ConfigSpace.Float('dropout_first', bounds=(0, 0.9), default=0.34, distribution=ConfigSpace.Normal(mu=0.5, sigma=0.35)),
            "dropout_second": ConfigSpace.Float('dropout_second', bounds=(0, 0.9), default=0.34, distribution=ConfigSpace.Normal(mu=0.5, sigma=0.35)),
            "weight_decay": ConfigSpace.Float("weight_decay", bounds=(0, 2), default=0.1),
            "n_blocks": ConfigSpace.UniformIntegerHyperparameter("n_blocks", lower=4, upper=5, default_value=5),
            "d_main": ConfigSpace.OrdinalHyperparameter("d_main", sequence=[256,512], default_value=512),
            "d_hidden" : ConfigSpace.OrdinalHyperparameter("d_hidden", sequence=[ 256, 512], default_value=512)
        },
    )

    dehb = DEHB(space, metric='mean_F1',min_budget=8, max_budget=250, rs=rs)

    start_time = time.process_time()

    print(f"Best configuration  {dehb.optimize(obj, limit=50,  unit='min', dataset=current_dataset)}")
    print(f"Time elapsed (CPU time): {(time.process_time() - start_time):.4f} seconds")
    print(dehb.inc_score)


Chosen dataset is  birds  (id:40588)
Original dataset shape is  (645, 279)
Number of predictors :  260 Number of targets :  19
1  numeric features were transformed into binary
Removed  0  parameters with one unique value
Removed  0  parameters with 1 occurance of value
Removed  0  parameters with pseudocorrelated values
Removed  128  parameters with correlated values


{'d_in': 143, 'n_blocks': 4, 'd_main': 512, 'd_hidden': 256, 'dropout_first': 0.3353363621058919, 'dropout_second': 0.10310874613762411, 'd_out': 19, 'batch_size': 256, 'epoch': 15, 'weight_decay': 1.726206851751187}
0.027767328538063425


{'d_in': 143, 'n_blocks': 5, 'd_main': 512, 'd_hidden': 512, 'dropout_first': 0.49538119076215653, 'dropout_second': 0.8377374000639999, 'd_out': 19, 'batch_size': 256, 'epoch': 15, 'weight_decay': 1.2465962536551158}
0.03495422193483135


{'d_in': 143, 'n_blocks': 4, 'd_main': 512, 'd_hidden': 256, 'dropout_first': 0.43003127542882263, 'dropout_second': 0.8568935677327879, 'd_out': 19, 'batch_size': 256, 'epoch': 15, 'weight_decay': 0.6617960497052984}
0.03615958587859248


{'d_in': 143, 'n_blocks': 5, 'd_main': 256, 'd_hidden': 256, 'dropout_first': 0.3251023841344367, 'dropout_second': 0.7047907898390531, 'd_out': 19, 'batch_size': 256, 'epoch': 15, 'weight_decay': 0.12711670057204727}
0.027634743408600667


...

{'d_in': 143, 'n_blocks': 5, 'd_main': 512, 'd_hidden': 512, 'dropout_first': 0.17669877790298927, 'dropout_second': 0.5643500096045972, 'd_out': 19, 'batch_size': 256, 'epoch': 62, 'weight_decay': 0.8779428414112722}
0.034682104960742734


{'d_in': 143, 'n_blocks': 4, 'd_main': 512, 'd_hidden': 512, 'dropout_first': 0.4936239508355729, 'dropout_second': 0.12900575224853322, 'd_out': 19, 'batch_size': 256, 'epoch': 62, 'weight_decay': 0.15691276268453191}
0.025411830310462093


{'d_in': 143, 'n_blocks': 4, 'd_main': 512, 'd_hidden': 512, 'dropout_first': 0.4599303993767613, 'dropout_second': 0.47218267171027356, 'd_out': 19, 'batch_size': 256, 'epoch': 125, 'weight_decay': 1.2701873017352876}
0.02684404423968852


{'d_in': 143, 'n_blocks': 5, 'd_main': 256, 'd_hidden': 512, 'dropout_first': 0.14397614332315062, 'dropout_second': 0.4594363055778315, 'd_out': 19, 'batch_size': 256, 'epoch': 125, 'weight_decay': 0.09060801954408904}
0.03294889821904703


{'d_in': 143, 'n_blocks': 5, 'd_main': 256, 'd_hidden': 512, 'dropout_first': 0.5789134025079993, 'dropout_second': 0.44870398162945757, 'd_out': 19, 'batch_size': 256, 'epoch': 250, 'weight_decay': 0.7722052756015485}
0.04035577188667825
TERMINAtion
TIME elapsed: 163.01245188713074
limit: 60
Best configuration  {'d_hidden': 256, 'd_main': 256, 'dropout_first': 0.22072049661777932, 'dropout_second': 0.4921028693685879, 'lr': 0.00022738055735631828, 'n_blocks': 4, 'weight_decay': 0.9875911927287815, 'budget': 15}
Time elapsed (CPU time): 77.0570 seconds
0.06273166036857368


In [ ]:
dehb.inc_config

{'d_hidden': 256,
 'd_main': 512,
 'dropout_first': 0.44716087482514294,
 'dropout_second': 0.1712805630460114,
 'lr': 0.00012260057359187555,
 'n_blocks': 4,
 'weight_decay': 1.774425485152653,
 'budget': 15}

In [ ]:
#train best model again with train + validation


train_split = Split(x=current_dataset.train_pred_data,y=current_dataset.train_tar_data)
test_split = Split(x=current_dataset.test_pred_data,y=current_dataset.test_tar_data)


loss_fn = nn.BCEWithLogitsLoss()


n_inputs, n_outputs = current_dataset.train_pred_data.shape[1], current_dataset.train_tar_data.shape[1]

train_dataset = MyDataset(train_split)
test_dataset = MyDataset(test_split)


train_dataloader = DataLoader(train_dataset, batch_size=256)
test_dataloader = DataLoader(test_dataset, batch_size=256)


config = { "d_in":n_inputs,
"n_blocks":dehb.inc_config['n_blocks'] ,
"d_main": dehb.inc_config['d_main'],
"d_hidden":dehb.inc_config['d_hidden'] ,
"dropout_first": dehb.inc_config['dropout_first'],
"dropout_second": dehb.inc_config['dropout_second'],
"d_out":n_outputs}

model = rtdl.ResNet.make_baseline(**config).to(device)  


optimizer = torch.optim.AdamW(model.parameters(), lr= dehb.inc_config['lr'], weight_decay=dehb.inc_config['weight_decay'])
budget = dehb.inc_config['budget']
for t in range(budget):
  train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
score_test = test(test_dataloader,model, loss_fn,ftt=False)

print("Final Test F1: "+ str(score_test))

(483, 143)
(162, 143)
Final Test F1: 0.0317020621729763


Birds best config 
Best configuration  Configuration(values={
  'batch_size': 512,
  'd_hidden': 512,
  'd_main': 512,
  'dropout_first': 0.1313298666316309,
  'dropout_second': 0.893321509153172,
  'lr': 0.0005885030438145404,
  'n_blocks': 5,
  'weight_decay': 1.3577145790987095,
})

0.33484101594084664

image
Best configuration  Configuration(values={
  'batch_size': 512,
  'd_hidden': 256,
  'd_main': 256,
  'dropout_first': 0.2918100884129402,
  'dropout_second': 0.31609682783434834,
  'lr': 0.00040899758743869434,
  'n_blocks': 5,
  'weight_decay': 0.20364401159485068,
})
f1_score: 0.6979468113995667

enron
Best configuration  Configuration(values={  
  'batch_size': 512,
  'd_hidden': 512,
  'd_main': 512,
  'dropout_first': 0.48921216512890775,
  'dropout_second': 0.3280479839105606,
  'lr': 0.0007695393682412402,
  'n_blocks': 4,
  'weight_decay': 0.3702232586704518,
})

Time elapsed (CPU time): 2250.9658 seconds
0.2257109478769928

Best configuration  Configuration(values={
  'batch_size': 512,
  'd_hidden': 512,
  'd_main': 512,
  'dropout_first': 0.272715756283827,
  'dropout_second': 0.05949379170911524,
  'lr': 0.0009258646792713822,
  'n_blocks': 5,
  'weight_decay': 0.7332812066200931,
})

Time elapsed (CPU time): 2873.9166 seconds
0.7350919546997978

langlog  'epoch': 250,
Best configuration  Configuration(values={
  'dropout_first': 0.34125283174065396,
  'dropout_second': 0.6942134407283955,
  'weight_decay': 0.3967631469771825,
})

Time elapsed (CPU time): 2890.4874 seconds
0.07342817451699311


reuters 40594
{'d_in': 243, 'n_blocks': 5, 'd_main': 512, 'd_hidden': 512, 'dropout_first': 0.1400190408305153,
 'dropout_second': 0.005229411495629055, 'd_out': 7, 'batch_size': 512, 'epoch': 125, 
 'weight_decay': 2.6967112095782535}

f1_score: 0.7010991205598903

scene epoch 125
Best configuration  Configuration(values={
  'dropout_first': 0.08430729104528323,
  'dropout_second': 0.5063009386369715,
  'weight_decay': 1.660342003774594,
})

Time elapsed (CPU time): 4171.2697 seconds
0.8116745455898824

slashdot
Best configuration  Configuration(values={
  'dropout_first': 0.06713299053940118,
  'dropout_second': 0.5121384023995134,
  'weight_decay': 0.7054137921863408,
})

Time elapsed (CPU time): 3939.2768 seconds
0.4050558337667007
yeast
Best configuration  Configuration(values={
  'dropout_first': 0.6215387204074385,
  'dropout_second': 0.25041387358251294,
  'weight_decay': 1.4810282766872704,
})

Time elapsed (CPU time): 3388.5205 seconds
0.47214501064303616


# For testing


In [ ]:
# WORK IN PROGRESS
import rtdl
import wandb
seed = 42

loss_fn = nn.BCEWithLogitsLoss()

best_config_dict = {dataset_num_to_id(0) : 
                    dataset_num_to_id(1) :
                    dataset_num_to_id(2) :
                    dataset_num_to_id(3) :
                    dataset_num_to_id(4) :
                    dataset_num_to_id(5) :
                    dataset_num_to_id(6) :
                    dataset_num_to_id(7) :
                    dataset_num_to_id(8) :
                    dataset_num_to_id(9) :
                    }


for id in range(10):
  dataset = OpenMlDataset(dataset_num_to_id(id), threshold = 0.85, normalization_type="standard", split=0.8, seed=seed)
  dataset_name = dataset_number_to_name(id)
  name = "ResNet_Preprocess_" + dataset_name
  
  pred_data, _ = dataset.get_predictors()
  target_data, _ = dataset.get_targets()

  cross_validation = CrossValidation(pred_data=pred_data, target_data=target_data, split_num=5)

  n_inputs, n_outputs = pred_data.shape[1], target_data.shape[1]
  
  # To replace with 

  config = best_config_dict[dataset_num_to_id(id)]

  config["batch_size"] = 256

  model = rtdl.ResNet.make_baseline(**config).to(device)  

  print(config)
  
  # For cross validation
  score_list = []
  for i, (train_pred, train_tar, val_pred, val_tar) in enumerate(cross_validation):

      train_split = Split(x=train_pred,y=train_tar)
      test_split = Split(x=val_pred,y=val_tar)

      model = rtdl.ResNet.make_baseline(**config).to(device) 

      optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-05)

      train_dataset = MyDataset(train_split)
      test_dataset = MyDataset(test_split)


      train_dataloader = DataLoader(train_dataset, batch_size=512)
      test_dataloader = DataLoader(test_dataset, batch_size=512)

      for t in range(budget):
        train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
      
      score_test = test(test_dataloader,model, loss_fn,ftt=False)

      score_list.append(score_test)

      del model
      torch.cuda.empty_cache()

      print("f1_score: " + str(score_test))

    # Get average and std
    score_array = np.array(score_list)
    avg_score = np.mean(score_array)
    std_score = np.std(score_array)
    print("Cross validation score : average ", avg_score", standard deviation ", std_score)


    # Leave one out test
    test_pred = dataset.get_test_predictors()
    test_target = dataset.get_test_target()
    train_split = Split(x=pred_data,y=target_data)
    test_split = Split(x=test_pred,y=test_target)
    
    model = rtdl.ResNet.make_baseline(**config).to(device) 

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-05)

    train_dataset = MyDataset(train_split)
    test_dataset = MyDataset(test_split)


    train_dataloader = DataLoader(train_dataset, batch_size=512)
    test_dataloader = DataLoader(test_dataset, batch_size=512)

    for t in range(budget):
      train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
    
    score_test = test(test_dataloader,model, loss_fn,ftt=False)

    score_list.append(score_test)

    del model
    torch.cuda.empty_cache()

    print("f1_score: " + str(score_test))

